# Artigo 4 - This is a Disaster!

## Objetivo

Para esse quarto artigo, o objetivo é por em prática os conhecimentos da lição 4 que se trata de processamento de linguagem natural, tendo como objetivo riar uma nova inteligência artificial capaz de fazer um processamento de linguagem natural nso dados, que no caso se tem como objetivo identificar se um determinado tweet se trata de um desastre natural ou não.
fazer um modelo de processamento de linguagem natural, que seja capaz de indeitificar através de tweets se realmente é um tweet de um desastre natural ou não. Além da criação da IA em si, no final ela será disponibilizada na plataforma Hugging Face.

## Motivação

Por questões de segurança da população, é de interesse que se saiba se o tweet se trata de um desastre natural ou não, uma vez que se essa identificação ocorrer, se saberá que determinado lugar pode ser classificado como perigoso naquele momento.

## Requisitos

Para que se possa realizar esse modelo, é necessário os dados, que serão retirados da competição [Natural Language Processing with Disaster Tweets](https://www.kaggle.com/competitions/nlp-getting-started/overview) e das bibliotecas que serão usadas para sua manipulação e criação e treinamento do modelo, que pode ser encontradas na célula 1.

In [1]:
# Não serão instaladas poís o Kaggle já as possui, nesa caso serão apenas importadas

import pandas as pd #Usado para a manipulação do arquivo com os dados
import numpy as np  #Usado para a manipulação matemática que irá ajudar as métricas
from datasets import Dataset, DatasetDict #Usado para usar o formato de data frame próprio do Hugging Face
from transformers import AutoModelForSequenceClassification,AutoTokenizer #Usado para poder baixar o tokenizador e criaçào do modelo
from transformers import TrainingArguments,Trainer #Usado para criar os argumentos de treino e realizar o treinamento

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Dados

Agora pode se começar a trazer os dados para poder se analisar o que se tem, como demostrado ná célula 2.

Para não ter que usar a API do Kagle e configurar uma chave eles serão baixados manualmente e colcoados na base de dados desse arquivo.

In [2]:
path = '/kaggle/input/disaster'
train_csv = f'{path}/train.csv'
test_csv = f'{path}/test.csv'

Agora com as caminhos definidos pode-se usar o pandas para realizar a leitura do csv de treino e visualizar o que ele traz, como também será transformado os valores inteiros em float por que esse é o padrão que será necessário mais para frente no trinamento do modelo, como demostrado na célula 3.

In [3]:
train= pd.read_csv(train_csv)
train = train.astype({'target':'float'})
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1.0
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1.0
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1.0
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1.0


Uma das primerias coisas que podem ser observadas é que o dataframe possui uma boa quantidade de dados, quase chegando a 10 mil linhas, ou seja, isso indica que possa haver uma grande variedade interessante para os treinos.

Para uma visualização ainda melhor, na célula 4 pode se observar a forma como esses dados estão dispostos no conjunto.

In [4]:
train.describe(include='object')

,keyword,location,text
count,7552,5080,7613
unique,221,3341,7503
top,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...
freq,45,104,10


Pode se observar que existem bastante dados repetidos, principalmente nas palavras chaves, mas isso já era esperado.

Agora com os dados visualizados, pode-se ver que a coluna target já está em formato numérico, coisa que será necessário, porém existem colunas com o valor NaN (Not a Number) que talvez tenham a necessidade de serem tradas posteriormente.

Mas em um primeiro momento, será criado apenas uma nova coluna chamada input que armazenará apenas o text, que é a principal fonte de informação para decidir se algo foi um desastre ou não, que pode ser observado na célula 5.

In [5]:
train['input'] = 'TEXT: ' + train['text']

train.input.head()

0    TEXT: Our Deeds are the Reason of this #earthq...
1         TEXT: Forest fire near La Ronge Sask. Canada
2    TEXT: All residents asked to 'shelter in place...
3    TEXT: 13,000 people receive #wildfires evacuat...
4    TEXT: Just got sent this photo from Ruby #Alas...
Name: input, dtype: object

Com isso já etá pronto para começar a trata-lo para poder ser utilizado para o treinamento do modelo.

### Preparando para o Modelo

Para essa parte será feito a seleção do modelo a ser usado e com isso será feito a tokenização e a  referente a como o modelo trabalha, para que ele possa interpretar os dados.

O primeiro é transformar em um dataset do hugging face, que pode ser obervado na célula 6.

In [6]:
train_ds = Dataset.from_pandas(train)

train_ds

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'target', 'input'],
    num_rows: 7613
})

Agora chegou a hora de selecionar o modelo, para isso, por questões de conveniência, será usado o mesmo modelo apresentada na lição 4 d fastai, porém podem ser encontrados outros modelos no Hugging Face.

Esse processo pode ser obervado na célula 7.

In [7]:
model_nm= 'microsoft/deberta-v3-small'

Agora será baixado os detalhes do modelo, como sua tokenização, e aplicado em uma linha do dataset para oder observar seu funcionamento, que pode ser obervado na célula 8.

In [8]:
tokz = AutoTokenizer.from_pretrained(model_nm)

tokz.tokenize("Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['▁Our',
 '▁Deeds',
 '▁are',
 '▁the',
 '▁Reason',
 '▁of',
 '▁this',
 '▁#',
 'earthquake',
 '▁May',
 '▁ALL',
 'AH',
 '▁Forgive',
 '▁us',
 '▁all']

Pode se observar que esse tokenizador está separando as palavras por '_'.

Desse modo, o texto do dataset pode ser representado no vocabulário do modelo que se deseja usar. Agora deve ser feito a numeralização de toda a coluna input, para que seja atribuido um idex para cada palavra para que se possa repreesentar elas de forma numérica e se passar para o modelo, que pode ser obsrevado na célula 9, através de uma função.

In [9]:
def toke_func(x): return tokz(x["input"])

tokz_train = train_ds.map(toke_func, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

Agora pode se visualizar como ficaram todos os dados tokenizados e depois numeralizados ná célula 10.

In [10]:
linha = tokz_train[0]
linha['input'], linha['input_ids']

('TEXT: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 [1,
  54453,
  294,
  581,
  65453,
  281,
  262,
  18037,
  265,
  291,
  953,
  117831,
  903,
  4924,
  17018,
  43632,
  381,
  305,
  2])

Agora antes que se possa começar a definir a métrica e o treinamento do modelo, deve-se lembrar que os modelos do Hugging Face precisam que o campo que se deseja prever se chame labels para o treino, esse processo pode ser observado na célula 11.

In [11]:
tokz_train = tokz_train.rename_columns({'target':'labels'})

### Métrica
Como não se tem uma métrica definida na competição, vai ser usado a de Pearson que é apresentada na lição 4 do fastai, que pode ser observado na lição 12.

In [12]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

### Divisão dos Dados
Para a última etapa dos dados, é necessário fazer a divisão do que será utilizado para treino e do que será utilizado para validação, uma vez também que os dados de teste já estào em um csv,

Para isso, deverá ocorrer uma divisão nos dados tratados para serem usados uma parte para treino e outro para validação, e por fim utilizar o dados de teste para comprovar que o modelo funciona.

Todo esse processo pode ser observado na célula 13.

In [13]:
train_dtv = tokz_train.train_test_split(0.20, seed=42)
train_dtv

DatasetDict({
    train: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1523
    })
})

## Treinamento
Agora pode se realizar o treinamento do modelo, para isso será utilizado as funções ja importadas anteriormente, sendo algumas valores como batch, epochs e learning rate foram inspiradas na lição 4 do fastai, uma vez que foi demostrado na lição que a escolha desses valores é arbritário, como pode ser observado na célula 14 e 15.

In [14]:
bs = 128
epochs = 4
lr = 8e-5

In [15]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

Agora que os argumentos de treino estào definidos, pode-se de fato criar o modelo e traina-lo, passando o dataset de treino e validação, como poder ser observado na célula 16, e por fim realizar de fato o treinamento como mostrado na célula 17.

In [16]:
model_prepared = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model_prepared, args, train_dataset=train_dtv['train'], eval_dataset=train_dtv['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.125121,0.700062
2,No log,0.121053,0.714383
3,No log,0.125702,0.710924
4,No log,0.133350,0.701830


TrainOutput(global_step=96, training_loss=0.13481924931208292, metrics={'train_runtime': 115.1558, 'train_samples_per_second': 211.54, 'train_steps_per_second': 0.834, 'total_flos': 447729660395100.0, 'train_loss': 0.13481924931208292, 'epoch': 4.0})

## Preparando o Teste
Agora para se usar o modelo treinado nos dados de teste é necessário fazer a mesma preparação com o csv de teste, que pode ser observado na célula 18.

In [18]:
test = pd.read_csv(test_csv)
test['input'] = 'TEXT: ' + test['text']
test_ds = Dataset.from_pandas(test)
test_ds_tk = test_ds.map(toke_func, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

## Predição
Agora pode-se usar a função para a predição dos dados de teste, que pode ser encontrada ná célula 19.

In [19]:
preds = trainer.predict(test_ds_tk).predictions.astype(int)
preds[:15]

array([[1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

## Deploy
Agora com o modelo pronto, pode se fazer sua exportação e coloca-lo no Hugging Face, onde foi feito um processo de se fazer upload através de alguns comandos, em um espaço particular e linkado com o espaço da disciplina.

O deploy pode ser encontrado em [Disaster](https://huggingface.co/spaces/fastaioncampus/Disaster)